In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

model_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
# model.eval()

In [10]:
def predict_mask(text, top_k=5):
    inputs = tokenizer(text, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    mask_token_logits = logits[0, mask_token_index, :]
    top_tokens = torch.topk(mask_token_logits, top_k, dim=1).indices[0].tolist()

    return [tokenizer.decode([token]).strip() for token in top_tokens]


In [11]:
import re

def predict_synonym_in_context(sentence, target_word, top_k=5):
    # Заменяем первое вхождение слова на [MASK]
    pattern = rf"\b{re.escape(target_word)}\b"
    masked_sentence = re.sub(pattern, tokenizer.mask_token, sentence, count=1)

    if tokenizer.mask_token not in masked_sentence:
        raise ValueError("Слово для замены не найдено в предложении.")

    print("С предложением:", masked_sentence)
    return predict_mask(masked_sentence, top_k=top_k)


In [12]:
predict_synonym_in_context("Он громко сказал и ушёл.", "сказал", top_k=5)

С предложением: Он громко [MASK] и ушёл.


['кричал', 'встал', 'поднялся', 'сказал', 'пошутил']

In [13]:
def predict_contextual_insert(masked_sentence, top_k=5):
    if tokenizer.mask_token not in masked_sentence:
        raise ValueError("В предложении должен быть [MASK].")

    print("С предложением:", masked_sentence)
    return predict_mask(masked_sentence, top_k=top_k)

In [14]:
predict_contextual_insert("Он [MASK] бежал.", top_k=5)

С предложением: Он [MASK] бежал.


['также', 'немедленно', 'тоже', 'не', 'снова']